# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [25]:
import sys
!{sys.executable} -m pip install -U pandas


In [26]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [27]:
# Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
   api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [28]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ["products", "customers", "orders"], # I choose a retail table since it's a common use case.
        'definition':["Product catalog: product_id, name, category, price",
                      "Customer directory: customer_id, name, city.",
                      "Orders/sales transactions: order_id, customer_id, product_id, quantity, order_date."
                      ],
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0   products  Product catalog: product_id, name, category, p...
1  customers       Customer directory: customer_id, name, city.
2     orders  Orders/sales transactions: order_id, customer_...


In [11]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [12]:
print(text_tables)

products: Product catalog: product_id, name, category, price
customers: Customer directory: customer_id, name, city.
orders: Orders/sales transactions: order_id, customer_id, product_id, quantity, order_date.


In [14]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""


In [15]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="How many orders were placed each day?")

In [16]:
print(return_OAI(pqt1))

```json
{
    "tables": ["orders"]
}
```


In [17]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="List all the customers in a given city")

In [18]:
print(return_OAI(pqt3))

{
    "tables": ["customers"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [21]:
import pandas as pd
# This is based of a database that might be used for a music streaming app for a mobile phone.
data = {
    "table": ["users", "subscriptions", "listening_history", "songs", "artists"],
    "definition": [
        "User directory: user_id, name, country, signup_date.",
        "Subscription status per user: user_id, plan (free/premium), start_date, end_date, monthly_price.",
        "Listening events: user_id, song_id, listen_timestamp, seconds_listened.",
        "Song catalog: song_id, title, artist_id, genre, duration_seconds.",
        "Artist directory: artist_id, artist_name, country."
    ]
}

df = pd.DataFrame(data)
print(df)
# apply the same table + row split logic
text_tables = "\n".join([f"{row['table']}: {row['definition']}" for _, row in df.iterrows()])
print("\n--- text_tables ---")
print(text_tables)


               table                                         definition
0              users  User directory: user_id, name, country, signup...
1      subscriptions  Subscription status per user: user_id, plan (f...
2  listening_history  Listening events: user_id, song_id, listen_tim...
3              songs  Song catalog: song_id, title, artist_id, genre...
4            artists  Artist directory: artist_id, artist_name, coun...

--- text_tables ---
users: User directory: user_id, name, country, signup_date.
subscriptions: Subscription status per user: user_id, plan (free/premium), start_date, end_date, monthly_price.
listening_history: Listening events: user_id, song_id, listen_timestamp, seconds_listened.
songs: Song catalog: song_id, title, artist_id, genre, duration_seconds.
artists: Artist directory: artist_id, artist_name, country.


In [22]:
# Use a strict template to improve results
# + you are a database table router, to help narrow down the intended use.
# + min set of tables required, make sure the model doesn't assume to much.
# used ONLY, JSON and  EXSACTLY top enphasis focus on result intent.
prompt_question_tables = """
You are a database table router.

Given the available tables and their content definitions:

### Tables
{tables}

### Task
Select the MINIMUM set of tables required to answer the user question.
Return ONLY valid JSON in EXACTLY this format:
{{"tables": ["table1", "table2"]}}

If no tables apply, return:
{{"tables": []}}

### User Question
{question}
"""


In [24]:
# place the questions in a list of strings to prompt test faster across multiple prompts and run it through a for loop.
questions = [
    # 1-table
    "How many users are there in total?",
    "List all artists from Canada.",

    # 2-table
    "How many premium users do we have?",
    "What are the top 5 most played songs by number of listens?",

    # 3-table
    "Which country generated the highest subscription revenue last month?",
    "Top 5 artists by total listening time (seconds listened).",

    # 4-table+
    "Which genre is most popular this month?",
    "Which new songs are trending the most right now?"
]

for q in questions:
    prompt = prompt_question_tables.format(tables=text_tables, question=q)
    print("\nQ:", q)
    print(return_OAI(prompt))





Q: How many users are there in total?
{"tables": ["users"]}

Q: List all artists from Canada.
{"tables": ["artists"]}

Q: How many premium users do we have?
{"tables": ["subscriptions"]}

Q: What are the top 5 most played songs by number of listens?
{"tables": ["listening_history", "songs"]}

Q: Which country generated the highest subscription revenue last month?
{"tables": ["users", "subscriptions"]}

Q: Top 5 artists by total listening time (seconds listened).
{"tables": ["listening_history", "artists"]}

Q: Which genre is most popular this month?
{"tables": ["listening_history", "songs"]}

Q: Which new songs are trending the most right now?
{"tables": ["songs", "listening_history"]}
